# Chapter 4: Reflection (代码示例)

本 notebook 演示了如何使用 LangChain 实现 Reflection（反思）模式，通过迭代循环不断改进代码。

## 前置要求

- Python 3.8+
- 阿里云 DashScope API Key（需要设置环境变量 `DASHSCOPE_API_KEY`）


## 1. 安装依赖

首先安装所需的 Python 包。


In [4]:
# 安装必要的依赖包（使用 %pip 在 Jupyter 中更推荐）
%pip install -q langchain-core>=0.1.0 langchain-community>=0.1.0 dashscope>=1.17.0 python-dotenv>=1.0.0

Note: you may need to restart the kernel to use updated packages.


## 2. 获取 API Key

如果环境变量或 .env 文件中没有设置 `DASHSCOPE_API_KEY`，可以在此处直接设置。


In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

# 获取 API key（优先从环境变量读取，如果没有则从 .env 文件）
dashscope_api_key = os.getenv("DASHSCOPE_API_KEY")
if not dashscope_api_key:
    print("警告：未找到 DASHSCOPE_API_KEY，请确保已设置环境变量或 .env 文件")


## 3. 导入库并初始化模型


In [6]:
from langchain_community.chat_models import ChatTongyi
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

# --- Configuration ---
# 初始化 Qwen 语言模型（使用阿里云 DashScope API）
# model 参数可以指定不同的 Qwen 模型，如 "qwen-flash", "qwen-turbo", "qwen-plus", "qwen-max" 等
# 对于代码生成和反思任务，建议使用 qwen-plus 或 qwen-max 以获得更好的推理能力
try:
    llm = ChatTongyi(
        model="qwen-plus",  # 使用 qwen-plus 以获得更好的代码生成和反思能力
        temperature=0.1,  # 较低的温度用于更确定性和聚焦的输出
        dashscope_api_key=dashscope_api_key
    )
    print(f"Language model initialized: qwen-plus")
except Exception as e:
    print(f"Error initializing language model: {e}")
    llm = None


def run_reflection_loop():
    """
    Demonstrates a multi-step AI reflection loop to progressively improve a Python function.
    """
    if not llm:
        print("LLM 未初始化，无法运行反思循环。请检查 API key 配置。")
        return

    # --- The Core Task ---
    task_prompt = """
    Your task is to create a Python function named `calculate_factorial`.
    This function should do the following:
    1.  Accept a single integer `n` as input.
    2.  Calculate its factorial (n!).
    3.  Include a clear docstring explaining what the function does.
    4.  Handle edge cases: The factorial of 0 is 1.
    5.  Handle invalid input: Raise a ValueError if the input is a negative number.
    """

    # --- The Reflection Loop ---
    max_iterations = 3
    current_code = ""
    # We will build a conversation history to provide context in each step.
    message_history = [HumanMessage(content=task_prompt)]

    for i in range(max_iterations):
        print("\n" + "="*25 + f" REFLECTION LOOP: ITERATION {i + 1} " + "="*25)

        # --- 1. GENERATE / REFINE STAGE ---
        # In the first iteration, it generates. In subsequent iterations, it refines.
        if i == 0:
            print("\n>>> STAGE 1: GENERATING initial code...")
            # The first message is just the task prompt.
            response = llm.invoke(message_history)
            current_code = response.content
        else:
            print("\n>>> STAGE 1: REFINING code based on previous critique...")
            # The message history now contains the task, the last code, and the last critique.
            # We instruct the model to apply the critiques.
            message_history.append(HumanMessage(content="Please refine the code using the critiques provided."))
            response = llm.invoke(message_history)
            current_code = response.content

        print("\n--- Generated Code (v" + str(i + 1) + ") ---\n" + current_code)
        message_history.append(response) # Add the generated code to history

        # --- 2. REFLECT STAGE ---
        print("\n>>> STAGE 2: REFLECTING on the generated code...")

        # Create a specific prompt for the reflector agent.
        # This asks the model to act as a senior code reviewer.
        reflector_prompt = [
            SystemMessage(content="""
                You are a senior software engineer and an expert in Python.
                Your role is to perform a meticulous code review.
                Critically evaluate the provided Python code based on the original task requirements.
                Look for bugs, style issues, missing edge cases, and areas for improvement.
                If the code is perfect and meets all requirements, respond with the single phrase 'CODE_IS_PERFECT'.
                Otherwise, provide a bulleted list of your critiques.
            """),
            HumanMessage(content=f"Original Task:\n{task_prompt}\n\nCode to Review:\n{current_code}")
        ]

        critique_response = llm.invoke(reflector_prompt)
        critique = critique_response.content

        # --- 3. STOPPING CONDITION ---
        if "CODE_IS_PERFECT" in critique:
            print("\n--- Critique ---\nNo further critiques found. The code is satisfactory.")
            break

        print("\n--- Critique ---\n" + critique)
        # Add the critique to the history for the next refinement loop.
        message_history.append(HumanMessage(content=f"Critique of the previous code:\n{critique}"))

    print("\n" + "="*30 + " FINAL RESULT " + "="*30)
    print("\nFinal refined code after the reflection process:\n")
    print(current_code)


# 运行反思循环示例
# 在 Jupyter notebook 中，可以直接运行此函数
if llm:
    run_reflection_loop()
else:
    print("LLM 未初始化，无法运行示例。请检查 API key 配置。")


Language model initialized: qwen-plus

========================= REFLECTION LOOP: ITERATION 1 =========================

>>> STAGE 1: GENERATING initial code...



--- Generated Code (v1) ---
```python
def calculate_factorial(n):
    """
    Calculate the factorial of a non-negative integer.
    
    The factorial of a number n (denoted as n!) is the product of all positive
    integers less than or equal to n. By definition, 0! = 1.
    
    Args:
        n (int): A non-negative integer for which to calculate the factorial.
        
    Returns:
        int: The factorial of n (n!).
        
    Raises:
        ValueError: If n is negative.
        TypeError: If n is not an integer.
        
    Examples:
        >>> calculate_factorial(5)
        120
        >>> calculate_factorial(0)
        1
        >>> calculate_factorial(3)
        6
    """
    # Check if input is an integer
    if not isinstance(n, int):
        raise TypeError("Input must be an integer")
    
    # Handle negative numbers
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    
    # Base case: 0! = 1
    if n == 0 or n == 1:
       